<a href="https://colab.research.google.com/github/rezvaya/DANN_/blob/master/DANN_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [3]:
#%cd /content/gdrive/

%cd /content/gdrive/My\ Drive/Diplom

/content/gdrive/My Drive/Diplom


In [0]:
import torch.nn as nn
from torch.autograd import Function
import torch.utils.data as data
from PIL import Image
#import Image
import os
import random
import torch.utils.data
import torch.backends.cudnn as cudnn
import torch.optim as optim
#from data_loader import GetLoader
from torchvision import datasets
from torchvision import transforms
#from model import CNNModel
import numpy as np
#from test import testG
import time
from PIL import Image
from torchvision import datasets, models, transforms

In [0]:
def simple_conv_block(in_channels, 
                      out_channels, 
                      kernel_size, 
                      stride, 
                      padding,
                      pool_size, 
                      pool_stride):
        return nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding),
            nn.ReLU(),
            nn.BatchNorm2d(out_channels),
            nn.MaxPool2d(pool_size, pool_stride))

In [0]:
def get_feature_extractor():
    feature = nn.Sequential()
    feature.add_module('conv1', simple_conv_block(3, 32, 10, 1, 1, 2, 2))
    feature.add_module('conv2', simple_conv_block(32, 64, 7, 1, 1, 2, 2))
    feature.add_module('conv3', simple_conv_block(64, 128, 5, 1, 1, 2, 2))
    feature.add_module('conv4', simple_conv_block(128, 256, 3, 1, 1, 2, 2))
    feature.add_module('conv5', simple_conv_block(256, 512, 3, 1, 1, 2, 2))
    return feature

In [0]:
def get_target_classifier():
    clf = nn.Sequential()
    clf.add_module('c_fc1', nn.Linear(81920,32))
    clf.add_module('c_bn1', nn.BatchNorm1d(32))
    clf.add_module('c_relu1', nn.ReLU(True))
    clf.add_module('c_drop1', nn.Dropout2d())
    clf.add_module('c_fc2', nn.Linear(32, 32))
    clf.add_module('c_bn2', nn.BatchNorm1d(32))
    clf.add_module('c_relu2', nn.ReLU(True))
    clf.add_module('c_fc3', nn.Linear(32, 7))
    clf.add_module('c_softmax', nn.LogSoftmax(dim=1))
    return clf

In [0]:
def get_domain_classifier():
    domain_clf = nn.Sequential()
    domain_clf.add_module('d_fc1', nn.Linear(81920,32))
    domain_clf.add_module('d_bn1', nn.BatchNorm1d(32))
    domain_clf.add_module('d_relu1', nn.ReLU(True))
    domain_clf.add_module('d_fc2', nn.Linear(32, 2))
    domain_clf.add_module('d_softmax', nn.LogSoftmax(dim=1))
    return domain_clf


In [9]:
feature_extractor = models.mobilenet_v2(pretrained=True).features

Downloading: "https://download.pytorch.org/models/mobilenet_v2-b0353104.pth" to /root/.cache/torch/checkpoints/mobilenet_v2-b0353104.pth


In [0]:
class ReverseLayerF(Function):
    @staticmethod
    def forward(ctx, x, alpha):
        ctx.alpha = alpha
        return x.view_as(x)

    @staticmethod
    def backward(ctx, grad_output):
        output = grad_output.neg() * ctx.alpha
        return output, None


class DomainTransferNet(nn.Module):
    def __init__(self, feature_extractor, classifier, domain_classifier):
        super(DomainTransferNet, self).__init__()
        self.feature_extractor = feature_extractor
        self.classifier = classifier
        self.domain_classifier = domain_classifier
        

    def forward(self, input_data, alpha):
        features = self.feature_extractor(input_data)
        features = features.view(features.size(0), -1)
        reverse_features = ReverseLayerF.apply(features, alpha)
        class_output = self.classifier(features)
        domain_output = self.domain_classifier(reverse_features)
        return class_output, domain_output
    

class GetLoader(data.Dataset):
    def __init__(self, data_root, data_list, transform=None):
        self.root = data_root
        self.transform = transform
        f = open(data_list, 'r')
        data_list = f.readlines()
        f.close()
        self.n_data = len(data_list)
        self.img_paths = []
        self.img_labels = []
        for data in data_list:
            self.img_paths.append(data[:-3])
            self.img_labels.append(data[-2])

    def __getitem__(self, item):
        img_paths, labels = self.img_paths[item], self.img_labels[item]
        imgs = Image.open(os.path.join(self.root, img_paths)).convert('RGB')

        if self.transform is not None:
            imgs = self.transform(imgs)
            labels = int(labels)

        return imgs, labels

    def __len__(self):
        return self.n_data

In [0]:
def testG(dataset_name, epoch):
    assert dataset_name in ['Source', 'RightPdd']

    model_root = 'models'
    image_root = os.path.join('./', dataset_name)

    cuda = True
    cudnn.benchmark = True
    batch_size = 128
    image_size = 256
    alpha = 0

    """load data"""

    img_transform_source = transforms.Compose([
        transforms.Resize(image_size),
        transforms.ToTensor(),
        transforms.Normalize([0.4352, 0.5103, 0.2836], [0.2193, 0.2073, 0.2047])
    ])

    img_transform_target = transforms.Compose([
        transforms.Resize(image_size),
        transforms.ToTensor(),
        transforms.Normalize([0.4352, 0.5103, 0.2836], [0.2193, 0.2073, 0.2047]) 
    ])

    if dataset_name == 'RightPdd':
        test_list = os.path.join(image_root, 'testPDD.txt')

        dataset = GetLoader(
            data_root=os.path.join(image_root, 'test'),
            data_list=test_list,
            transform=img_transform_target
        )
    else:
        test_list = os.path.join(image_root, 'testpVil.txt')

        dataset = GetLoader(
            data_root=os.path.join(image_root, 'test'),
            data_list=test_list,
            transform=img_transform_target
        )

    dataloader = torch.utils.data.DataLoader(
        dataset=dataset,
        batch_size=batch_size,
        shuffle=False,
        num_workers=8
    )

    """ training """

    my_net = torch.load(os.path.join(
        model_root, 'PDD_transfer_net_epoch_' + str(epoch//10) + '.pth'
    ))
    my_net = my_net.eval()

    if cuda:
        my_net = my_net.cuda()

    len_dataloader = len(dataloader)
    data_target_iter = iter(dataloader)

    i = 0
    n_total = 0
    n_correct = 0


    while i < len_dataloader:

        # test model using target data
        data_target = data_target_iter.next()
        t_img, t_label = data_target

        batch_size = len(t_label)

        input_img = torch.FloatTensor(batch_size, 3, image_size, image_size)
        class_label = torch.LongTensor(batch_size)

        if cuda:
            t_img = t_img.cuda()
            t_label = t_label.cuda()
            input_img = input_img.cuda()
            class_label = class_label.cuda()

        input_img.resize_as_(t_img).copy_(t_img)
        class_label.resize_as_(t_label).copy_(t_label)

        class_output, _ = my_net(input_data=input_img, alpha=alpha)
        pred = class_output.data.max(1, keepdim=True)[1]
        n_correct += pred.eq(class_label.data.view_as(pred)).cpu().sum()
        n_total += batch_size

        i += 1

    accu = n_correct.data.numpy() * 1.0 / n_total

    print ('epoch: %d, accuracy of the %s dataset: %f' % (epoch, dataset_name, accu))


In [12]:
source_dataset_name = 'Source'
target_dataset_name = 'RightPdd'
source_image_root = os.path.join('./', source_dataset_name)
target_image_root = os.path.join('./', target_dataset_name)
model_root = 'models'
cuda = True
cudnn.benchmark = True
lr = 0.0009

batch_size = 128
#batch_size = 256
image_size = 256
n_epoch = 500
num_classes=7

manual_seed = random.randint(1, 10000)
random.seed(manual_seed)
torch.manual_seed(manual_seed)

# load data
target_classifier=get_target_classifier()
domain_classifier=get_domain_classifier()

img_transform_source = transforms.Compose([
    transforms.Resize(image_size),
    transforms.ToTensor(),
    transforms.Normalize([0.4352, 0.5103, 0.2836], [0.2193, 0.2073, 0.2047])
   

])

img_transform_target = transforms.Compose([
    transforms.Resize(image_size),
    transforms.ToTensor(),
    transforms.Normalize([0.4352, 0.5103, 0.2836], [0.2193, 0.2073, 0.2047])   
])

train_list_source = os.path.join(source_image_root, 'trainpVil.txt')

dataset_source = GetLoader(
    data_root=os.path.join(source_image_root, 'train'),
    data_list=train_list_source,
    transform=img_transform_source
)

dataloader_source = torch.utils.data.DataLoader(
    dataset=dataset_source,
    batch_size=batch_size,
    shuffle=True,
    num_workers=8)

train_list = os.path.join(target_image_root, 'trainPDD.txt')

dataset_target = GetLoader(
    data_root=os.path.join(target_image_root, 'train'),
    data_list=train_list,
    transform=img_transform_target
)

dataloader_target = torch.utils.data.DataLoader(
    dataset=dataset_target,
    batch_size=batch_size,
    shuffle=True,
    num_workers=8)

# load model



transfer_net=DomainTransferNet(
    feature_extractor,
    target_classifier, 
    domain_classifier
    )


# setup optimizer

optimizer = optim.Adam(transfer_net.parameters(), lr=lr, eps=1e-08, amsgrad=True)

loss_class = torch.nn.NLLLoss()
loss_domain = torch.nn.NLLLoss()

if cuda:
    transfer_net = transfer_net.cuda()
    loss_class = loss_class.cuda()
    loss_domain = loss_domain.cuda()

ct = 0
for layer in feature_extractor.children():
  ct = ct + 1
  if ct < 12:
      for param in layer.parameters():
          param.requires_grad = False
  else:
      for param in layer.parameters():
          param.requires_grad = True

#for p in transfer_net.feature_extractor.parameters():
#    p.requires_grad = False

for p in transfer_net.classifier.parameters():
    p.requires_grad = True

for p in transfer_net.domain_classifier.parameters():
    p.requires_grad = True

# training

for epoch in range(n_epoch):

    len_dataloader = min(len(dataloader_source),len(dataloader_target))
       # , 
    #print(len_dataloader)
    data_source_iter = iter(dataloader_source)
    data_target_iter = iter(dataloader_target)
    i = 0

    while i < len_dataloader:

        p = float(i + epoch * len_dataloader) / n_epoch / len_dataloader
        alpha = 2. / (1. + np.exp(-10 * p)) - 1

        # training model using source data

        data_source = data_source_iter.next()
        s_img, s_label = data_source

        transfer_net.zero_grad()
        batch_size = len(s_label)

        input_img = torch.FloatTensor(batch_size, 3, image_size, image_size)
        class_label = torch.LongTensor(batch_size)
        
        domain_label = torch.zeros(batch_size)
        domain_label = domain_label.long()

        if cuda:
            s_img = s_img.cuda()
            s_label = s_label.cuda()
            input_img = input_img.cuda()
            class_label = class_label.cuda()
            domain_label = domain_label.cuda()

        input_img.resize_as_(s_img).copy_(s_img)
        class_label.resize_as_(s_label).copy_(s_label)
        
        class_output, domain_output = transfer_net(input_img, alpha)
        
        err_s_label = loss_class(class_output, class_label)
        err_s_domain = loss_domain(domain_output, domain_label)

        # training model using target data
        data_target = data_target_iter.next()
        t_img, _ = data_target
        batch_size = len(t_img)

        input_img = torch.FloatTensor(batch_size, 3, image_size, image_size)
        domain_label = torch.ones(batch_size)
        domain_label = domain_label.long()

        if cuda:
            t_img = t_img.cuda()
            input_img = input_img.cuda()
            domain_label = domain_label.cuda()

        input_img.resize_as_(t_img).copy_(t_img)

        _, domain_output = transfer_net(input_img, alpha=alpha)
        err_t_domain = loss_domain(domain_output, domain_label)
        err = err_t_domain + err_s_domain + err_s_label

        err.backward()
        optimizer.step()

        i += 1

        print ('epoch: %d, [iter: %d / all %d], err_s_label: %f, err_s_domain: %f, err_t_domain: %f' \
              % (epoch, i, len_dataloader, err_s_label.data.cpu().numpy(),
                 err_s_domain.data.cpu().numpy(), err_t_domain.data.cpu().item()))
        
    number=epoch//10
    torch.save(transfer_net, '{0}/PDD_transfer_net_epoch_{1}.pth'.format(model_root, number))
    testG(source_dataset_name, epoch)
    testG(target_dataset_name, epoch)

print('done')


epoch: 0, [iter: 1 / all 3], err_s_label: 1.944443, err_s_domain: 0.679203, err_t_domain: 0.745973
epoch: 0, [iter: 2 / all 3], err_s_label: 1.544633, err_s_domain: 0.784052, err_t_domain: 0.786394
epoch: 0, [iter: 3 / all 3], err_s_label: 1.478314, err_s_domain: 0.667055, err_t_domain: 0.741957


/usr/local/lib/python3.6/dist-packages/torch/serialization.py:360: UserWarning: Couldn't retrieve source code for container of type DomainTransferNet. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


epoch: 0, accuracy of the Source dataset: 0.672925
epoch: 0, accuracy of the RightPdd dataset: 0.426471
epoch: 1, [iter: 1 / all 3], err_s_label: 1.424016, err_s_domain: 0.661312, err_t_domain: 0.728374
epoch: 1, [iter: 2 / all 3], err_s_label: 1.454828, err_s_domain: 0.652827, err_t_domain: 0.713981
epoch: 1, [iter: 3 / all 3], err_s_label: 1.363955, err_s_domain: 0.648040, err_t_domain: 0.709172
epoch: 1, accuracy of the Source dataset: 0.928854
epoch: 1, accuracy of the RightPdd dataset: 0.485294
epoch: 2, [iter: 1 / all 3], err_s_label: 1.342045, err_s_domain: 0.644527, err_t_domain: 0.703860
epoch: 2, [iter: 2 / all 3], err_s_label: 1.319809, err_s_domain: 0.652269, err_t_domain: 0.702744
epoch: 2, [iter: 3 / all 3], err_s_label: 1.288360, err_s_domain: 0.639687, err_t_domain: 0.703643
epoch: 2, accuracy of the Source dataset: 0.950593
epoch: 2, accuracy of the RightPdd dataset: 0.441176
epoch: 3, [iter: 1 / all 3], err_s_label: 1.289987, err_s_domain: 0.647144, err_t_domain: 0.69

KeyboardInterrupt: ignored